신경망을 구축할 때는 변화도를 0으로 만들어 주는 것이 좋다. 기본적으로 ```.backward()```를 호출할 때마다 변화도가 버퍼에 쌓이기 때문이다. (덮어쓰지 않는 다는 의미)

## 개요
신경망을 학습시킬 때, 경사 하강법을 거쳐 모델 정확도를 높일 수 있다. 경사 하강법은 간단히 설명해 모델의 가중치와 편향을 약간씩 수정하면서 손실(또는 오류)를 최소화하는 과정이다.<br><br>
```torch.Tensor```는 PyTorch의 핵심이 되는 클래스다. 텐서를 생성할 때 ```.requires_grad```속성을 ```True```로 설정하면, 텐서에 가해진 모든 연산을 추척한다. 뒤따르는 모든 역전파 단계에서도, 이 텐서의 변화도는 ```.grad``` 속성에 누적된다. 모든 변화도의 축적 또는 합은 손실 텐서에서 ```.backward()```를 호출할 때 계산된다. <br><br>
텐서의 변화도를 0으로 만들어 주어야 하는 경우도 있다. 예를 들어 학습 과정 반복문을 시작할 때, 누적되는 변화도를 정확하게 추적하기 위해서는 변화도를 우선 0으로 만들어줘야한다. 이 레시피에서는 PyTorch 라이브러리를 사용하여 변화도를 0으로 만드는 방법을 배운다. PyTorch에 내장된 ```CIFAR10```데이터셋에 대하여 신경망을 훈련시키는 과정을 통해 알아본다.

## 설정
이 레시피에는 데이터를 학습시키는 내용이 포함되어 있어 런타임을 GPU 또는 TPU로 전환하는 것이 좋다.
시작하기에 앞서, ```torch```와 ```torchvision```패키지가 없다면 설치한다

In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


## 단계
1단계부터 4단계까지는 학습을 위한 데이터와 신경망을 준비하며, 5단계에서 변화도를 0으로 만들어 준다.
1. 데이터를 불러오기 위해 필요한 라이브러리 import
2. 데이터셋 불러오고 정규화
3. 신경망 구축
4. 손실 함수 정의
5. 신경망을 학습시킬 때 변화도 0으로 만들기


### 1. 데이터를 불러오기 위해 필요한 라이브러리 import

In [4]:
import torch

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

### 2. 데이터셋 불러오고 정규화

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                         shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

2.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

5.3%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

7.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

10.1%IOPub me

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### 3. 신경망 구축하기

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### 4. 손실 함수와 옵티마이저 정의
분류를 위한 Cross-Entropy 손실 함수와 모멘텀을 설정한 SGD 옵티마이저를 사용

In [7]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### 5. 신경망을 학습시키는 동안 변화도를 0으로 만들기
여기서 할 일은 데이터 이터레이터를 순회하면서, 신경망에 입력을 주고 최적화하는 것이다.<br>
데이터의 엔티티 각각의 변화도를 0으로 만들어주는 것에 유의. 신경망을 학습시킬 때 불필요한 정보를 추적하지 않도록 하기 위함이다.

In [8]:
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

/Users/hongjin-u/opt/anaconda3/envs/Pytorch/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459064158/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss: 2.184
[1,  4000] loss: 1.801
[1,  6000] loss: 1.655
[1,  8000] loss: 1.569
[1, 10000] loss: 1.497
[1, 12000] loss: 1.459
[2,  2000] loss: 1.386
[2,  4000] loss: 1.337
[2,  6000] loss: 1.320
[2,  8000] loss: 1.311
[2, 10000] loss: 1.292
[2, 12000] loss: 1.279
Finished Training


```model.zero_grad()```를 사용해도 변화도를 0으로 만들 수 있다. 이는 옵티마이저에 모든 모델 파라미터가 포함되는 한 ```optimizer.zero_grad()```를 사용하는 것과 동일하다. 